In [2]:
import argparse
import numpy as np
import os
import tabulate
import torch
import torch.nn.functional as F

import load_data
import models
import curves
import utils

c_dir = './plane'
grid_points = 13 #number of points in the grid
margin_left = 0.2
margin_right = 0.2
margin_bottom = 0.2
margin_top = 0.2
curve_points = 61 # number of points on the curve
batch_size = 64
model_name = 'LSTMClassifier'
curve_type = 'PolyChain'
num_bends = 3
ckpt =  './saved_models/LSTMClassifier_curve_polychain4_5-44.pt'    # checkpoint to eval
wd = 1e-4

In [5]:
TEXT, vocab_size, num_classes, word_embeddings, train_loader, valid_loader, test_loader = \
                                load_data.load_dataset(batch_size=batch_size)

Length of Text Vocabulary: 135872
Vector size of Text Vocabulary:  torch.Size([135872, 300])
Label Length: 4


In [4]:
kwargs = {
    'batch_size': batch_size,
    'hidden_size': 256,
    'embedding_length': 300,
    'vocab_size': vocab_size,
    'weights': word_embeddings
}

torch.backends.cudnn.benchmark = True

architecture = getattr(models, model_name)
curve = getattr(curves, curve_type)

curve_model = curves.CurveNet(
    num_classes,
    curve,
    architecture.curve,
    num_bends,
    architecture_kwargs=kwargs,
)
curve_model.cuda()

checkpoint = torch.load(ckpt)
curve_model.load_state_dict(checkpoint['model_state'])

criterion = F.cross_entropy
regularizer = utils.l2_regularizer(wd)


def get_xy(point, origin, vector_x, vector_y):
    return np.array([np.dot(point - origin, vector_x), np.dot(point - origin, vector_y)])


w = list()
curve_parameters = list(curve_model.net.parameters())
for i in range(num_bends):
    w.append(np.concatenate([
        p.data.cpu().numpy().ravel() for p in curve_parameters[i::num_bends]
    ]))

print('Weight space dimensionality: %d' % w[0].shape[0])

u = w[2] - w[0]
dx = np.linalg.norm(u)
u /= dx

v = w[1] - w[0]
v -= np.dot(u, v) * u
dy = np.linalg.norm(v)
v /= dy

bend_coordinates = np.stack(get_xy(p, w[0], u, v) for p in w)

ts = np.linspace(0.0, 1.0, curve_points)
curve_coordinates = []
for t in np.linspace(0.0, 1.0, curve_points):
    weights = curve_model.weights(torch.Tensor([t]).cuda())
    curve_coordinates.append(get_xy(weights, w[0], u, v))
curve_coordinates = np.stack(curve_coordinates)

G = grid_points
alphas = np.linspace(0.0 - margin_left, 1.0 + margin_right, G)
betas = np.linspace(0.0 - margin_bottom, 1.0 + margin_top, G)

tr_loss = np.zeros((G, G))
tr_nll = np.zeros((G, G))
tr_acc = np.zeros((G, G))
tr_err = np.zeros((G, G))

te_loss = np.zeros((G, G))
te_nll = np.zeros((G, G))
te_acc = np.zeros((G, G))
te_err = np.zeros((G, G))

grid = np.zeros((G, G, 2))

base_model = architecture.base(num_classes=num_classes, **kwargs)
base_model.cuda()

columns = ['i', 'j', 'X', 'Y', 'Train loss', 'Train nll', 'Train error (%)', 'Test nll', 'Test error (%)']

for i, alpha in enumerate(alphas):
    for j, beta in enumerate(betas):
        p = w[0] + alpha * dx * u + beta * dy * v

        offset = 0
        for parameter in base_model.parameters():
            size = np.prod(parameter.size())
            value = p[offset:offset+size].reshape(parameter.size())
            parameter.data.copy_(torch.from_numpy(value))
            offset += size

        utils.update_bn(train_loader, base_model)

        tr_res = utils.eval_model(train_loader, base_model, criterion, batch_size, regularizer)
        te_res = utils.eval_model(test_loader, base_model, criterion, batch_size, regularizer)

        tr_loss_v, tr_nll_v, tr_acc_v = tr_res['loss'], tr_res['nll'], tr_res['acc']
        te_loss_v, te_nll_v, te_acc_v = te_res['loss'], te_res['nll'], te_res['acc']

        c = get_xy(p, w[0], u, v)
        grid[i, j] = [alpha * dx, beta * dy]

        tr_loss[i, j] = tr_loss_v
        tr_nll[i, j] = tr_nll_v
        tr_acc[i, j] = tr_acc_v
        tr_err[i, j] = 100.0 - tr_acc[i, j]

        te_loss[i, j] = te_loss_v
        te_nll[i, j] = te_nll_v
        te_acc[i, j] = te_acc_v
        te_err[i, j] = 100.0 - te_acc[i, j]

        values = [ i, j,
            grid[i, j, 0], grid[i, j, 1], tr_loss[i, j], tr_nll[i, j], tr_err[i, j],
            te_nll[i, j], te_err[i, j]
        ]
        table = tabulate.tabulate([values], columns, tablefmt='simple', floatfmt='10.4f')
        if j == 0:
            table = table.split('\n')
            table = '\n'.join([table[1]] + table)
        else:
            table = table.split('\n')[2]
        print(table)

np.savez(
    os.path.join(c_dir, 'plane4_5_polychain.npz'),
    ts=ts,
    bend_coordinates=bend_coordinates,
    curve_coordinates=curve_coordinates,
    alphas=alphas,
    betas=betas,
    grid=grid,
    tr_loss=tr_loss,
    tr_acc=tr_acc,
    tr_nll=tr_nll,
    tr_err=tr_err,
    te_loss=te_loss,
    te_acc=te_acc,
    te_nll=te_nll,
    te_err=te_err
)

Weight space dimensionality: 41334020


/home/hakobtamazyan/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:52: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


/home/hakobtamazyan/venv3/lib/python3.5/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/home/hakobtamazyan/venv3/lib/python3.5/site-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train), lengths



---  ---  ----------  ----------  ------------  -----------  -----------------  ----------  ----------------
  i    j           X           Y    Train loss    Train nll    Train error (%)    Test nll    Test error (%)
---  ---  ----------  ----------  ------------  -----------  -----------------  ----------  ----------------
  0    0    -41.5692    -33.5521        0.1778       0.0831             3.4109      0.4702           12.1765



  0    1    -41.5692    -13.9800        0.1466       0.0520             2.2051      0.5397           11.6975


...


 12    8    249.4150    123.0244        1.6951       1.6000            18.0590      2.2589           22.3866



 12    9    249.4150    142.5965        1.9077       1.8116            20.1844      2.4361           24.2605



 12   10    249.4150    162.1686        2.0659       1.9687            21.8369      2.5881           25.6471



 12   11    249.4150    181.7406        2.1622       2.0639            23.0326      2.6239           26.5294



 12   12    249.4150    201.3127        2.2041       2.1046            23.8783      2.6578           27.6891


In [6]:
c_dir = './plane'
grid_points = 15 #number of points in the grid
margin_left = 0.2
margin_right = 0.2
margin_bottom = 0.2
margin_top = 0.2
curve_points = 61 # number of points on the curve
batch_size = 128
model_name = 'LSTMClassifier'
curve_type = 'Bezier'
num_bends = 3
ckpt =  './saved_models/LSTMClassifier_curve_bezier4_5-42.pt'    # checkpoint to eval
wd = 1e-4

kwargs = {
    'batch_size': batch_size,
    'hidden_size': 256,
    'embedding_length': 300,
    'vocab_size': vocab_size,
    'weights': word_embeddings
}

torch.backends.cudnn.benchmark = True

architecture = getattr(models, model_name)
curve = getattr(curves, curve_type)

curve_model = curves.CurveNet(
    num_classes,
    curve,
    architecture.curve,
    num_bends,
    architecture_kwargs=kwargs,
)
curve_model.cuda()

checkpoint = torch.load(ckpt)
curve_model.load_state_dict(checkpoint['model_state'])

criterion = F.cross_entropy
regularizer = utils.l2_regularizer(wd)


def get_xy(point, origin, vector_x, vector_y):
    return np.array([np.dot(point - origin, vector_x), np.dot(point - origin, vector_y)])


w = list()
curve_parameters = list(curve_model.net.parameters())
for i in range(num_bends):
    w.append(np.concatenate([
        p.data.cpu().numpy().ravel() for p in curve_parameters[i::num_bends]
    ]))

print('Weight space dimensionality: %d' % w[0].shape[0])

u = w[2] - w[0]
dx = np.linalg.norm(u)
u /= dx

v = w[1] - w[0]
v -= np.dot(u, v) * u
dy = np.linalg.norm(v)
v /= dy

bend_coordinates = np.stack(get_xy(p, w[0], u, v) for p in w)

ts = np.linspace(0.0, 1.0, curve_points)
curve_coordinates = []
for t in np.linspace(0.0, 1.0, curve_points):
    weights = curve_model.weights(torch.Tensor([t]).cuda())
    curve_coordinates.append(get_xy(weights, w[0], u, v))
curve_coordinates = np.stack(curve_coordinates)

G = grid_points
alphas = np.linspace(0.0 - margin_left, 1.0 + margin_right, G)
betas = np.linspace(0.0 - margin_bottom, 1.0 + margin_top, G)

tr_loss = np.zeros((G, G))
tr_nll = np.zeros((G, G))
tr_acc = np.zeros((G, G))
tr_err = np.zeros((G, G))

te_loss = np.zeros((G, G))
te_nll = np.zeros((G, G))
te_acc = np.zeros((G, G))
te_err = np.zeros((G, G))

grid = np.zeros((G, G, 2))

base_model = architecture.base(num_classes=num_classes, **kwargs)
base_model.cuda()

columns = ['i', 'j', 'X', 'Y', 'Train loss', 'Train nll', 'Train error (%)', 'Test nll', 'Test error (%)']

for i, alpha in enumerate(alphas):
    for j, beta in enumerate(betas):
        p = w[0] + alpha * dx * u + beta * dy * v

        offset = 0
        for parameter in base_model.parameters():
            size = np.prod(parameter.size())
            value = p[offset:offset+size].reshape(parameter.size())
            parameter.data.copy_(torch.from_numpy(value))
            offset += size

        utils.update_bn(train_loader, base_model)

        tr_res = utils.eval_model(train_loader, base_model, criterion, batch_size, regularizer)
        te_res = utils.eval_model(test_loader, base_model, criterion, batch_size, regularizer)

        tr_loss_v, tr_nll_v, tr_acc_v = tr_res['loss'], tr_res['nll'], tr_res['acc']
        te_loss_v, te_nll_v, te_acc_v = te_res['loss'], te_res['nll'], te_res['acc']

        c = get_xy(p, w[0], u, v)
        grid[i, j] = [alpha * dx, beta * dy]

        tr_loss[i, j] = tr_loss_v
        tr_nll[i, j] = tr_nll_v
        tr_acc[i, j] = tr_acc_v
        tr_err[i, j] = 100.0 - tr_acc[i, j]

        te_loss[i, j] = te_loss_v
        te_nll[i, j] = te_nll_v
        te_acc[i, j] = te_acc_v
        te_err[i, j] = 100.0 - te_acc[i, j]

        values = [ i, j,
            grid[i, j, 0], grid[i, j, 1], tr_loss[i, j], tr_nll[i, j], tr_err[i, j],
            te_nll[i, j], te_err[i, j]
        ]
        table = tabulate.tabulate([values], columns, tablefmt='simple', floatfmt='10.4f')
        if j == 0:
            table = table.split('\n')
            table = '\n'.join([table[1]] + table)
        else:
            table = table.split('\n')[2]
        print(table)

np.savez(
    os.path.join(c_dir, 'plane4_5_bezier.npz'),
    ts=ts,
    bend_coordinates=bend_coordinates,
    curve_coordinates=curve_coordinates,
    alphas=alphas,
    betas=betas,
    grid=grid,
    tr_loss=tr_loss,
    tr_acc=tr_acc,
    tr_nll=tr_nll,
    tr_err=tr_err,
    te_loss=te_loss,
    te_acc=te_acc,
    te_nll=te_nll,
    te_err=te_err
)

Weight space dimensionality: 41334020


/home/hakobtamazyan/venv3/lib/python3.5/site-packages/ipykernel_launcher.py:66: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


/home/hakobtamazyan/venv3/lib/python3.5/site-packages/torchtext/data/field.py:322: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train)
/home/hakobtamazyan/venv3/lib/python3.5/site-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  return Variable(arr, volatile=not train), lengths



---  ---  ----------  ----------  ------------  -----------  -----------------  ----------  ----------------
  i    j           X           Y    Train loss    Train nll    Train error (%)    Test nll    Test error (%)
---  ---  ----------  ----------  ------------  -----------  -----------------  ----------  ----------------
  0    0    -41.5692    -32.1823        0.1638       0.0686             3.0640      0.4631           12.6333



  0    1    -41.5692    -16.0911        0.1417       0.0468             2.0966      0.5224           12.2500


...


 14   10    249.4150    128.7292        2.8875       2.7950            30.5270      3.1034           33.4167



 14   11    249.4150    144.8203        3.4573       3.3641            35.7340      3.6357           38.7000



 14   12    249.4150    160.9115        4.0046       3.9107            40.4881      4.1183           43.1167



 14   13    249.4150    177.0026        4.4504       4.3557            44.5609      4.5707           47.0667



 14   14    249.4150    193.0937        4.8262       4.7307            48.0351      4.9496           50.4333
